In [ ]:
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import glob

In [ ]:
# Download global fires from a defined time period
# Data from https://earthobservatory.nasa.gov/global-maps/MOD14A1_M_FIRE
data_dir = glob.glob(r'D:\notebooks\projects\globalfires\*.tiff')
countries = gpd.GeoDataFrame.from_file(r'D:\notebooks\projects\globalfires\countries.gpkg')
n = 0

# Empty dataframe to store results
data = pd.DataFrame()

# Read the raster
for fires in data_dir:
    n = n + 1
    print str(fires)
    firemonth = os.path.join(r'D:\notebooks\projects\globalfires', fires)
    fire = rasterio.open(firemonth)
    
    monthyear = str(firemonth[70:-11])
    year, month = str(monthyear[0:4]), str(monthyear[5:8])
    
    print year, month
    
    array = fire.read(1)
    affine = fire.transform
    
    # remove the no data values, .1 (water)
    array[array==99999] = 0
    array[array<1] = 0
    
    # Zonal stats - summarize the pixel counts by country
    fire_stats = zonal_stats(countries, array, affine=affine, stats=['count','sum', 'mean'])
    firestats = pd.DataFrame(fire_stats, columns={'count','sum','mean'})
    firestats['monthly_fires'] = firestats['sum']
    firestats['Month'] = month
    firestats['Year'] = year

    # add to dataframe
    data = data.append(firestats)

print str(n) + ' months summarized'
data.to_csv(os.path.join(r'D:\notebooks\projects\globalfires','country_fire_analysis.csv'))